# Running simulations through the cloud 

This notebook is a tutorial of the API used for submitting simulations to our servers.


In [1]:
import tidy3d as td
import tidy3d.web as web

# set logging level to ERROR because we'll only running simulations to demonstrate the web API, we dont care about the results.
td.config.logging_level = "ERROR"


## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions, note: if False (default) they will all run silently
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


11:15:25 Eastern Daylight Time Created task 'webAPI' with task_id               
                               'fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=22457;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=426983;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\taskId]8;;\
                               ]8;id=22457;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\=]8;;\]8;id=228768;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\fdve]8;;\]8;id=22457;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\-482d8ff4-1de8-4508-8e06-6431135b2c7f']8;;\.

Output()

11:15:27 Eastern Daylight Time status = queued

                               To cancel the simulation, use                    
                               'web.abort(task_id)' or 'web.delete(task_id)' or 
                               abort/delete the task in the web UI. Terminating 
                               the Python script will not stop the job running  
                               on the cloud.

...

11:16:00 Eastern Daylight Time status = postprocess

11:16:05 Eastern Daylight Time status = success

                               View simulation result at                        
                               ]8;id=637391;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=779214;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\taskId]8;;\
                               ]8;id=637391;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\=]8;;\]8;id=274433;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\fdve]8;;\]8;id=637391;https://tidy3d.simulation.cloud/workbench?taskId=fdve-482d8ff4-1de8-4508-8e06-6431135b2c7f\-482d8ff4-1de8-4508-8e06-6431135b2c7f']8;;\.

Output()

11:16:07 Eastern Daylight Time loading simulation from data/sim.hdf5

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.webapi.run.html) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.webapi.start.html) and then exit the session and load the results at a later time using [web.load](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.webapi.load.html).


## Job Container

The [web.Job](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.Simulation.html).

We can get the cost estimate of running the task before actually running it. This prevents us from accidentally running large jobs that we set up by mistake. The estimated cost is the maximum cost corresponding to running all the time steps.

In [4]:
# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# estimate the maximum cost
estimated_cost = web.estimate_cost(job.task_id)

                               Created task 'job' with task_id                  
                               'fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=588377;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=667123;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\taskId]8;;\
                               ]8;id=588377;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\=]8;;\]8;id=6628;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\fdve]8;;\]8;id=588377;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\-7016bdf8-0b42-4abf-ba9d-08195d373f92']8;;\.

Output()

11:16:08 Eastern Daylight Time Maximum FlexCredit cost: 0.025. Minimum cost     
                               depends on task execution details. Use           
                               'web.real_cost(task_id)' to get the billed       
                               FlexCredit cost after a simulation run.

While [Job](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [5]:
# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")

11:16:09 Eastern Daylight Time status = queued

                               To cancel the simulation, use                    
                               'web.abort(task_id)' or 'web.delete(task_id)' or 
                               abort/delete the task in the web UI. Terminating 
                               the Python script will not stop the job running  
                               on the cloud.

Output()

11:16:16 Eastern Daylight Time status = preprocess

11:16:17 Eastern Daylight Time Maximum FlexCredit cost: 0.025. Use              
                               'web.real_cost(task_id)' to get the billed       
                               FlexCredit cost after a simulation run.

...

11:16:21 Eastern Daylight Time status = postprocess

11:16:25 Eastern Daylight Time status = success

                               View simulation result at                        
                               ]8;id=264660;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=631587;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\taskId]8;;\
                               ]8;id=264660;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\=]8;;\]8;id=607127;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\fdve]8;;\]8;id=264660;https://tidy3d.simulation.cloud/workbench?taskId=fdve-7016bdf8-0b42-4abf-ba9d-08195d373f92\-7016bdf8-0b42-4abf-ba9d-08195d373f92']8;;\.

Output()

11:16:27 Eastern Daylight Time loading simulation from data/sim.hdf5

Another convenient thing about [Job](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [6]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

11:16:28 Eastern Daylight Time loading simulation from data/sim.hdf5

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.Batch.html) object.
The batch object is like a [Job](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.Job.html), but stores task metadata for a series of simulations.

In [7]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


                               Created task 'sim_0' with task_id                
                               'fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=808503;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=819926;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4\taskId]8;;\
                               ]8;id=808503;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4\=]8;;\]8;id=546893;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4\fdve]8;;\]8;id=808503;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4\-2e46f52d-77e8-4bba-87e4-64c648a132f4']8;;\.

Output()

11:16:29 Eastern Daylight Time Created task 'sim_1' with task_id                
                               'fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=622947;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=544162;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8\taskId]8;;\
                               ]8;id=622947;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8\=]8;;\]8;id=190861;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8\fdve]8;;\]8;id=622947;https://tidy3d.simulation.cloud/workbench?taskId=fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8\-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8']8;;\.

...

11:16:33 Eastern Daylight Time Started working on Batch.

11:16:34 Eastern Daylight Time Maximum FlexCredit cost: 0.075 for the whole     
                               batch.

                               Use 'Batch.real_cost()' to get the billed        
                               FlexCredit cost after the Batch has completed.

Output()

11:16:49 Eastern Daylight Time Batch complete.

When the batch is completed, the output is not a [SimulationData](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.BatchData.html).  The data within this [BatchData](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.container.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [8]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values[0])
    intensities[task_name] = sum_intensity

print(intensities)


Output()

11:16:51 Eastern Daylight Time loading simulation from                          
                               data\fdve-2e46f52d-77e8-4bba-87e4-64c648a132f4.hd
                               f5

Output()

11:16:52 Eastern Daylight Time loading simulation from                          
                               data\fdve-4f2afb7b-d93b-489c-b5b9-60b312b9b0a8.hd
                               f5

Output()

11:16:53 Eastern Daylight Time loading simulation from                          
                               data\fdve-bf9497b0-bbc0-4ebf-a090-92d3805d0380.hd
                               f5

{'sim_0': 6377911.0, 'sim_1': 6377911.0, 'sim_2': 6377911.0}


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](https://docs.flexcompute.com/projects/tidy3d/en/latest/api/_autosummary/tidy3d.web.api.asynchronous.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [9]:
batch_results = web.run_async(simulations=sims, verbose=verbose)


                               Created task 'sim_0' with task_id                
                               'fdve-3e163014-4be9-4d47-91b0-490e44a64f19' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=888349;https://tidy3d.simulation.cloud/workbench?taskId=fdve-3e163014-4be9-4d47-91b0-490e44a64f19\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=846180;https://tidy3d.simulation.cloud/workbench?taskId=fdve-3e163014-4be9-4d47-91b0-490e44a64f19\taskId]8;;\
                               ]8;id=888349;https://tidy3d.simulation.cloud/workbench?taskId=fdve-3e163014-4be9-4d47-91b0-490e44a64f19\=]8;;\]8;id=862887;https://tidy3d.simulation.cloud/workbench?taskId=fdve-3e163014-4be9-4d47-91b0-490e44a64f19\fdve]8;;\]8;id=888349;https://tidy3d.simulation.cloud/workbench?taskId=fdve-3e163014-4be9-4d47-91b0-490e44a64f19\-3e163014-4be9-4d47-91b0-490e44a64f19']8;;\.

Output()

11:16:54 Eastern Daylight Time Created task 'sim_1' with task_id                
                               'fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=127949;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=86180;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c\taskId]8;;\
                               ]8;id=127949;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c\=]8;;\]8;id=960451;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c\fdve]8;;\]8;id=127949;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c\-21d2a378-4508-4c6b-b50c-f9181fdfa91c']8;;\.

...

11:16:59 Eastern Daylight Time Started working on Batch.

11:17:01 Eastern Daylight Time Maximum FlexCredit cost: 0.075 for the whole     
                               batch.

                               Use 'Batch.real_cost()' to get the billed        
                               FlexCredit cost after the Batch has completed.

Output()

11:17:14 Eastern Daylight Time Batch complete.

In [10]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values[0])
    intensities[task_name] = sum_intensity

print(intensities)


Output()

11:17:15 Eastern Daylight Time loading simulation from                          
                               .\fdve-3e163014-4be9-4d47-91b0-490e44a64f19.hdf5

Output()

11:17:17 Eastern Daylight Time loading simulation from                          
                               .\fdve-21d2a378-4508-4c6b-b50c-f9181fdfa91c.hdf5

Output()

11:17:18 Eastern Daylight Time loading simulation from                          
                               .\fdve-7cbf20ca-4312-4220-8ff0-deffb7c240ae.hdf5

{'sim_0': 6377911.0, 'sim_1': 6377911.0, 'sim_2': 6377911.0}


After going through this tutorial, you have learned how to run simulations with Tidy3D via the web API. If you are new to Tidy3D or the finite-difference time-domain (FDTD) method, we highly recommend going through our [FDTD101](https://www.flexcompute.com/fdtd101/) tutorials and [example library](https://www.flexcompute.com/tidy3d/examples/) before starting your own simulation adventure. 